# Greeks Coberture for options
* The objetive of this proyect is make a coberture 

Using the finance option **AMAZON**


In [1]:
## Import Packages
import pandas as pd
import numpy as np

**MAKING an data analysis**

In [2]:
## Load data 
data= pd.read_excel('Proyecto Griegas .xlsx')

In [3]:
#data.describe()

**EDA**
> Removing Variables that not need 

* we need only the adj close in Bid y Ask, and stock price 

In [4]:
df=data[['Last Trade Date','Strike','Bid', 'Ask','Stock price']]
df['Adj Close']=df[['Bid','Ask']].mean(axis=1)
df['Last Trade Date']=df['Last Trade Date'].astype(str).str[:10]
df['Last Trade Date']=df['Last Trade Date'].apply(lambda x: pd.to_datetime(x,format='%Y/%m/%d'))
df.set_index('Last Trade Date',inplace=True)

<ipython-input-4-5a4c3c3fe163>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Adj Close']=df[['Bid','Ask']].mean(axis=1)
<ipython-input-4-5a4c3c3fe163>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Last Trade Date']=df['Last Trade Date'].astype(str).str[:10]
<ipython-input-4-5a4c3c3fe163>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

## Definiremos nuestra r basandonos en la TIE 
* Tomando datos historicos del año pasado **por BANXICO**


In [5]:
data_TIE= pd.read_excel('Proyecto Griegas .xlsx','TIE',)
data_TIE=data_TIE.iloc[16:,:2]
data_TIE.columns=['Fecha','TIE']
TIE=round(data_TIE['TIE'].mean(),2)
print('La TIE promedio en el ultimo año es:',TIE,'%')

La TIE promedio en el ultimo año es: 9.35 %


## Definimos los parametros a trabajar
* Necesitamos los datos historicos de 1 año atras sobre la accion

**Objetivo:**
> * Obtener mu
>* Obtener Sigma

In [6]:
##Chargin data
Historical_Data= pd.read_excel('Proyecto Griegas .xlsx','Parámetros')
Historical_Data=Historical_Data.iloc[2:,4:6].copy()
##trating the data
Historical_Data.columns=('Fecha','St')
Historical_Data['St']=Historical_Data['St'].astype(float)
##Log return
Historical_Data['ln(St)']=np.log(Historical_Data['St'].astype(float))
Historical_Data[['St','ln(St)']]=Historical_Data[['St','ln(St)']].round(2)
Historical_Data['Fecha']=Historical_Data['Fecha'].apply(lambda x: pd.to_datetime(x,format='%Y/%m/%d'))
Historical_Data.set_index('Fecha',inplace=True)
Historical_Data.sort_index(ascending=False,inplace=True)
##Returns
Historical_Data['%_Change']=round(Historical_Data['St'].pct_change(),5)
Historical_Data['Log_Return']=round(np.log(Historical_Data.St) - np.log(Historical_Data.St.shift(1)),5)
##Eda
Historical_Data.dropna(inplace=True)
Historical_Data.sort_index(ascending=True,inplace=True)
Historical_Data.head()

* Encontrando los parametros

In [11]:
S=Historical_Data['St']
Tau,mu,sigma,r,K=[1,
                  round(np.mean(Historical_Data['ln(St)']),2),
                  round(np.var(Historical_Data['ln(St)']),2),
                  round(TIE/100,2),
                  90]
print('Nuestros Parámetros son:\n''Tau:',Tau,'\nmu:',mu,'\nsigma:',sigma,'\nr:',r,'\nK:',K)

Nuestros Parámetros son:
Tau: 1 
mu: 4.67 
sigma: 0.02 
r: 0.09 
K: 90


# Greeks

**Llamamos una libreria propia para continuar con las griegas**

In [12]:
import Libreria_Propia as Lb

* d1

In [13]:
parametros=[S,K,r,sigma,Tau,K] ##Llamamos funcion para poder llamar correctamente las variables
d1=Lb.d1(parametros)

* Delta

In [14]:
Delta_Call=Lb.Call_Delta(d1)

* Gamma

In [15]:
Gamma_params=[sigma,S,Tau,d1]
Gamma_Call=Lb.Call_Gamma(Gamma_params)

* Vega

In [16]:
Vega_params=[S,Tau,d1]
Vega_Call=Lb.Call_Vega(Vega_params)

## Unimos todo en nuestro df inicial

In [17]:
Historical_Data['Delta \u0394']=Delta_Call
Historical_Data['Gamma \u0393']=Gamma_Call
Historical_Data['Vega v']=Vega_Call
Historical_Data.head()

,St,ln(St),%_Change,Log_Return,Delta Δ,Gamma Γ,Vega v
Fecha,,,,,,,
2022-05-09,108.79,4.69,-0.00064,-0.00064,1.0,2.296995e-44,1.362881e-41
2022-05-10,108.86,4.69,0.03312,0.03258,1.0,1.462991e-44,8.691571e-42
2022-05-11,105.37,4.66,-0.01459,-0.01470,1.0,3.349401e-35,1.864322e-32
2022-05-12,106.93,4.67,-0.05422,-0.05574,1.0,2.794027e-39,1.601584e-36
2022-05-13,113.06,4.73,0.02031,0.02010,1.0,6.978391e-57,4.471912e-54


## Creando la cobertura con nuestros 3 esenarios
1. Sin griega
2. Con Delta $\Delta$
3. Con Delta Gamma $\Gamma$
4. Con Delta Gamma Vega **V**